# Loading data

In [1]:
import warnings
warnings.filterwarnings('ignore')
from tensorboard.plugins.hparams import api as hp

# Dataset hyper parameters
HP_BALANCED = hp.HParam('balanced', hp.Discrete([0, 1])) # 0 unbalanced, 1 balanced
HP_NORM_IMAGES = hp.HParam('norm_images', hp.Discrete([0, 1]))

# Network hyper parameters
HP_NETWORKS = hp.HParam('network', hp.Discrete(['Xception',
                                            'VGG16',
                                            'VGG19',
                                            'ResNet50',
                                            'ResNet50V2',
                                            'ResNet101',
                                            'ResNet101V2',
                                            'ResNet152',
                                            'ResNet152V2',
                                            'InceptionV3',
                                            'InceptionResNetV2',
                                            'MobileNet',
                                            'MobileNetV2',
                                            'DenseNet121',
                                            'DenseNet169',
                                            'DenseNet201',
                                            'NASNetMobile',
                                            'NASNetLarge',
                                            'EfficientNetB0',
                                            'EfficientNetB1',
                                            'EfficientNetB2',
                                            'EfficientNetB3',
                                            'EfficientNetB4',
                                            'EfficientNetB5',
                                            'EfficientNetB6',
                                            'EfficientNetB7',
                                            'EfficientNetV2B0',
                                            'EfficientNetV2B1',
                                            'EfficientNetV2B2',
                                            'EfficientNetV2B3',
                                            'EfficientNetV2S',
                                            'EfficientNetV2M',
                                            'EfficientNetV2L',
                                            'ConvNeXtTiny',
                                            'ConvNeXtSmall',
                                            'ConvNeXtBase',
                                            'ConvNeXtLarge',
                                            'ConvNeXtXLarge' ])) 


HP_ACTIVATION_FUNCTIONS = hp.HParam('act_functions', hp.Discrete(['relu', 'selu', 'tanh']))
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 32, 64, 128]))
HP_EARLY_STOP = hp.HParam('early_stop', hp.Discrete([3, 5, 7, 10, 15]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.0, 0.5))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

# METRIC_ACCURACY = 'accuracy'


# hparams = [
#      HP_BALANCED,
#      HP_NORM_IMAGES,
#      HP_NETWORKS,
#      HP_ACTIVATION_FUNCTIONS,
#      HP_NUM_UNITS,
#      HP_DROPOUT,
#      HP_EARLY_STOP,
#      HP_OPTIMIZER
# ]

# metrics = [hp.Metric(x, group='train', display_name=f'{x} (train)') for x in METRICS_NAMES] + \
#           [hp.Metric(x, group='validation', display_name=f'{x} (validation)') for x in METRICS_NAMES]


# with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
#     hp.hparams_config(
#         hparams=hparams,
#         # metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
#         metrics = metrics
#       )

2022-10-26 01:58:53.513598: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-26 01:58:53.677392: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-26 01:58:53.711609: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-26 01:58:54.346962: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:
import tensorflow as tf
import numpy as np

np.random.seed(42)
tf.random.set_seed(42)


def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)
        result_metrics = train_test_model(hparams)
        tf.summary.scalar(METRIC_NAMES, result_metrics, step=1)

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation 
from pyts.image import MarkovTransitionField
import dask.dataframe as dd  
import pandas as pd
    

from scipy import signal as sign
from PIL import Image
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import matthews_corrcoef

import warnings
warnings.filterwarnings('ignore')

In [4]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

# Function for normalizing images
def normImages(X):
    for i, image in enumerate(X):
        max_n = np.max(image)
        image /= max_n
        X[i] = np.abs(image)
    return X

# Obtaining training, validation and test data
def train_val_test_split(balanced, normalized):
    spectrogram = ['spectrogram', 'mel', 'mtf']
    
    X_train = []
    X_test = []
    X_val = []

    y_flag = 0 # y's has not been obtained yet 
    # root_dir = "/home/polivares/scratch/Datasets/PowerLineFaults/"
    root_dir = '/home/polivares/Dropbox/Work/PostDoc/PowerLineFaults/'
    
    for sp in spectrogram:
        X_full = np.load(f"{root_dir}dataset_{sp}/full/images_full.npy")[:3000]
        
        
        if not y_flag:
            y_flag = 1
            
            y_full = np.load(f"{root_dir}dataset_{sp}/full/labels_full.npy").reshape(-1)[:3000]
            if balanced: # getting balanced data from index
                # Index 1, partial discharge
                index_1 = np.where(y_full==1)[0]
                len_index_1 = len(index_1)
                index_train_1, index_val_1, index_test_1 = index_1[:len_index_1//3], index_1[len_index_1//3:2*len_index_1//3], index_1[2*len_index_1//3:]

                # Index 0, non partial discharge
                index_0 = np.where(y_full==0)[0]
                index_train_0, index_val_0, index_test_0 = index_0[:len_index_1//3], index_0[len_index_1//3:2*len_index_1//3], index_0[2*len_index_1//3:]

                # Obtaining index
                index_train = np.concatenate([index_train_0, index_train_1])
                np.random.shuffle(index_train)
                index_val = np.concatenate([index_val_0, index_val_1])
                np.random.shuffle(index_val)
                index_test = np.concatenate([index_test_0, index_test_1])
                np.random.shuffle(index_test)

            else: # Unbalanced data, similar to the original from index
                index_full = np.arange(len(y_full))
                np.random.shuffle(index_full)
                len_index = 1000
                
                # Obtaining index
                index_train, index_val, index_test = index_full[:len_index], index_full[len_index:2*len_index], index_full[2*len_index:]
            
            y_train = y_full[index_train]
            y_val = y_full[index_val]
            y_test = y_full[index_test]
            
            del y_full
        
        if normalized:
            X_full = normImages(X_full)
        
        X_train.append(X_full[index_train])
        X_val.append(X_full[index_val])
        X_test.append(X_full[index_test])
        
        del X_full
        
    X_train_c = np.concatenate(X_train, axis=3) 
    X_val_c = np.concatenate(X_val, axis=3)
    X_test_c = np.concatenate(X_test, axis=3)
    
    return X_train_c, y_train, X_val_c, y_val, X_test_c, y_test


# X_train_c, y_train, X_val_c, y_val, X_test_c, y_test = train_val_test_split(balanced=0,normalized=0)

In [5]:
from tensorflow.keras.callbacks import EarlyStopping

def train_test_model(hparams, METRICS):
    # Model creation
    print("Model creation")
    base_model = getattr(tf.keras.applications, hparams[HP_NETWORKS])(include_top=False, weights='imagenet', input_shape=(256, 256, 3))
    x = tf.keras.layers.Conv2D(64, (3,3), activation=hparams[HP_ACTIVATION_FUNCTIONS])(base_model.output)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2,2))(x)
    x = tf.keras.layers.Dropout(hparams[HP_DROPOUT])(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=hparams[HP_ACTIVATION_FUNCTIONS])(x)
    x = tf.keras.layers.Dropout(hparams[HP_DROPOUT])(x)
    # avg = tf.keras.layers.GlobalAveragePooling2D()()
    output = tf.keras.layers.Dense(1, activation="sigmoid")(x)
    model = tf.keras.Model(inputs=base_model.input, outputs=output)

    for layer in base_model.layers:
        layer.trainable = False # Esto impide que las capas se re entrenen
    
    # Compile model
    print("Model compile")
    early_stopping = EarlyStopping(patience = hparams[HP_EARLY_STOP])
    model.compile(loss = 'bce', optimizer = hparams[HP_OPTIMIZER], metrics=METRICS)
    
    # Obtaining data 
    print("Obtaining data")
    X_train, y_train, X_val, y_val, X_test, y_test = train_val_test_split(hparams[HP_BALANCED], hparams[HP_NORM_IMAGES])
    
    # Fitting training
    print("Fitting training")
    history_model = model.fit(X_train, y_train, epochs=1000, 
                            validation_data=(X_val, y_val),
                            batch_size=10,
                            callbacks=[early_stopping])
    # Evaluation on test
    print("Evaluation on test")
    results = model.evaluate(X_test, y_test)
    
    # Returning metrics results
    print("Returning metrics results")
    return results

In [8]:
# Run evaluation with hparams
def run(run_dir, hparams):
    METRICS = [
          tf.keras.metrics.TruePositives(name='tp'),
          tf.keras.metrics.FalsePositives(name='fp'),
          tf.keras.metrics.TrueNegatives(name='tn'),
          tf.keras.metrics.FalseNegatives(name='fn'), 
          tf.keras.metrics.BinaryAccuracy(name='accuracy'),
          tf.keras.metrics.Precision(name='precision'),
          tf.keras.metrics.Recall(name='recall'),
          tf.keras.metrics.AUC(name='auc'),
          tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
    ]
    
    METRICS_NAMES = [
        'loss',
        'tp',
        'fp',
        'tn',
        'fn',
        'accuracy',
        'precision',
        'recall',
        'auc',
        'prc'
    ]

    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        results = train_test_model(hparams, METRICS)
    
        for name, metric in zip(METRICS_NAMES, results):
            print(f"Summary: metric {name} value {metric}")
            tf.summary.scalar(name, metric, step=1)
    # return results

In [10]:
session_num = 0

for balanced in HP_BALANCED.domain.values:
    for norm_image in HP_NORM_IMAGES.domain.values:
        for network in HP_NETWORKS.domain.values:
            for act_func in HP_ACTIVATION_FUNCTIONS.domain.values:
                for num_units in HP_NUM_UNITS.domain.values:
                    for dropout in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
                        for early_stop in HP_EARLY_STOP.domain.values:
                            for optimizer in HP_OPTIMIZER.domain.values:
                                hparams = {
                                     HP_BALANCED : balanced,
                                     HP_NORM_IMAGES: norm_image,
                                     HP_NETWORKS: network,
                                     HP_ACTIVATION_FUNCTIONS: act_func,
                                     HP_NUM_UNITS: num_units,
                                     HP_DROPOUT: dropout,
                                     HP_EARLY_STOP: early_stop,
                                     HP_OPTIMIZER: optimizer
                                }
                                run_name = "run-%d" % session_num
                                print('--- Starting trial: %s' % run_name)
                                print({h.name: hparams[h] for h in hparams})
                                run('logs/hparam_tuning/' + run_name, hparams)
                                session_num += 1
                                break
                            break
                        break
                    break
                break
            break
        break
    break


--- Starting trial: run-0
{'balanced': 0, 'norm_images': 0, 'network': 'ConvNeXtBase', 'act_functions': 'relu', 'num_units': 16, 'dropout': 0.0, 'early_stop': 3, 'optimizer': 'adam'}
Model creation
Model compile
Obtaining data
Fitting training
Epoch 1/1000
100/100 [==============================] - 30s 247ms/step - loss: 0.4366 - tp: 10.0000 - fp: 33.0000 - tn: 897.0000 - fn: 60.0000 - accuracy: 0.9070 - precision: 0.2326 - recall: 0.1429 - auc: 0.6473 - prc: 0.1346 - val_loss: 0.2742 - val_tp: 18.0000 - val_fp: 98.0000 - val_tn: 843.0000 - val_fn: 41.0000 - val_accuracy: 0.8610 - val_precision: 0.1552 - val_recall: 0.3051 - val_auc: 0.7746 - val_prc: 0.1459
Epoch 2/1000
100/100 [==============================] - 23s 229ms/step - loss: 0.2161 - tp: 21.0000 - fp: 23.0000 - tn: 907.0000 - fn: 49.0000 - accuracy: 0.9280 - precision: 0.4773 - recall: 0.3000 - auc: 0.8207 - prc: 0.2938 - val_loss: 0.1940 - val_tp: 23.0000 - val_fp: 39.0000 - val_tn: 902.0000 - val_fn: 36.0000 - val_accuracy

In [ ]:
results

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

spectrogram = 'mel' # 'spectrogram', 'mel', 'mfcc', 'mtf'

# # root_dir = "/home/polivares/scratch/Datasets/PowerLineFaults/"
root_dir = '/home/polivares/Dropbox/Work/PostDoc/PowerLineFaults/'

X_full_2 = np.load(f"{root_dir}dataset_{spectrogram}/full/images_full.npy")
# y_full_2 = np.load(f"{root_dir}dataset_{spectrogram}/full/labels_full.npy").reshape(-1)

In [ ]:
X_full_2 = normImages(X_full_2)

X_train_2 = X_full_2[index_train]
X_val_2 = X_full_2[index_val]
X_test_2 = X_full_2[index_test]

del X_full_2

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

spectrogram = 'mtf' # 'spectrogram', 'mel', 'mfcc', 'mtf'

# # root_dir = "/home/polivares/scratch/Datasets/PowerLineFaults/"
root_dir = '/home/polivares/Dropbox/Work/PostDoc/PowerLineFaults/'

X_full_3 = np.load(f"{root_dir}dataset_{spectrogram}/full/images_full.npy")
# y_full_2 = np.load(f"{root_dir}dataset_{spectrogram}/full/labels_full.npy").reshape(-1)

In [ ]:
X_full_3 = normImages(X_full_3)

X_train_3 = X_full_3[index_train]
X_val_3 = X_full_3[index_val]
X_test_3 = X_full_3[index_test]

del X_full_3

In [ ]:
X_train_c = np.concatenate((X_train, X_train_2, X_train_3), axis=3)
X_val_c = np.concatenate((X_val, X_val_2, X_val_3), axis=3)
X_test_c = np.concatenate((X_test[:1000], X_test_2[:1000], X_test_3[:1000]), axis=3)
del X_train, X_train_2, X_train_3, X_val, X_val_2, X_val_3, X_test, X_test_2, X_test_3
X_train_c.shape

# CNN-model

In [ ]:
# Creación de modelo
model = models.Sequential()
# Capas encargadas de obtener información de la imagen
model.add(layers.Conv2D(256, (3, 3), activation='selu', input_shape=(256,256,3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='selu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='selu'))
# Capas para la clasificación en base a la información obtenida en 
# capas anteriores
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='selu'))
model.add(layers.Dense(1, activation = 'sigmoid'))
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(patience = 7)

checkpoint_path='CNN-model_mtf.ckpt'
# Create a callback that saves the model's weights
# cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
#                                                  save_weights_only=True,
#                                                  verbose=1)

METRICS = [
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
      tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

model.compile(loss = 'bce', optimizer = 'adam', metrics=METRICS)
history_cnn = model.fit(X_train_c, y_train, epochs=100, 
                        validation_data=(X_val_c, y_val),
                        batch_size=10,
                        callbacks=[early_stopping, 
#                                   cp_callback
                                  ]
                       )

In [ ]:
plt.plot(history_cnn.history['accuracy'], label='accuracy')
plt.plot(history_cnn.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')
plt.show()

In [ ]:
plt.plot(history_cnn.history['loss'], label='loss')
plt.plot(history_cnn.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='lower right')
plt.show()

In [ ]:
val_loss, val_acc, val_recall = model.evaluate(X_val,  y_val, verbose=2, batch_size=4)

In [ ]:
val_predictions = (model.predict(X_val, batch_size=4)>0.5).reshape(1,-1)[0].astype(int)
val_real_values = y_val.T[0]

In [ ]:
matthews_corrcoef(val_real_values, val_predictions)

In [ ]:
test_loss, test_acc, test_recall = model.evaluate(X_test,  y_test, verbose=2, batch_size=4)

In [ ]:
test_predictions = (model.predict(X_test, batch_size=4)>0.5).reshape(1,-1)[0].astype(int)
test_real_values = y_test.T[0]

In [ ]:
matthews_corrcoef(test_real_values, test_predictions)

# CNN 2

In [ ]:
# # Creación de modelo
# model = models.Sequential()
# # Capas encargadas de obtener información de la imagen
# model.add(layers.Conv2D(256, (3, 3), activation='selu', input_shape=(256,256,1),kernel_initializer='random_normal',))
# model.add(layers.MaxPooling2D((2, 2)))
# # model.add(layers.Dropout(0.3))
# model.add(layers.Conv2D(128, (3, 3), activation='selu',kernel_initializer='random_normal',))
# model.add(layers.MaxPooling2D((2, 2)))
# # model.add(layers.Dropout(0.3))
# model.add(layers.Conv2D(64, (3, 3), activation='selu',kernel_initializer='random_normal',))
# # Capas para la clasificación en base a la información obtenida en 
# # capas anteriores
# model.add(layers.Flatten())
# model.add(layers.Dense(64, activation='selu',kernel_initializer='random_normal',))
# model.add(layers.Dense(1, activation = 'sigmoid',kernel_initializer='random_normal',))
# model.summary()

In [ ]:
# Creación de modelo
# Armado de nuestra red neuronal
model = models.Sequential()
model.add(layers.Conv2D(32, (5, 5), input_shape=(256, 256, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.5))
model.add(layers.Conv2D(64, (5, 5), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.5))
model.add(layers.Conv2D(128, (5, 5), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.5))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

In [ ]:
METRICS = [
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
      tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(patience = 10, restore_best_weights=True)

model.compile(loss='binary_crossentropy',
# optimizer=tf.keras.optimizers.RMSprop(lr=0.001),
metrics=METRICS)

In [ ]:
history = model.fit(X_train_c, y_train,
batch_size=10,
epochs=100,
verbose=1,
validation_data=(X_val_c, y_val),
callbacks=[early_stopping]
)

In [ ]:
result = model.evaluate(X_test_c[:1000],  y_test[:1000], verbose=2, batch_size=4)

In [ ]:
test_predictions = (model.predict(X_test, batch_size=4)>0.5).reshape(1,-1)[0].astype(int)
test_real_values = y_test.T[0]

del X_train_c, X_val_c

# Transfer Learning

In [ ]:
# Transfer Learning
# base_model = tf.keras.applications.vgg16.VGG16(include_top=False, weights='imagenet', input_shape=(256, 256, 3))
base_model = tf.keras.applications.EfficientNetB7(include_top=False, weights='imagenet', input_shape=(256, 256, 3))
x = tf.keras.layers.Conv2D(64, (3,3), activation='relu')(base_model.output)
x = tf.keras.layers.MaxPooling2D(pool_size=(2,2))(x)
# x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(100, activation='relu')(x)
# x = tf.keras.layers.Dropout(0.2)(x)
# avg = tf.keras.layers.GlobalAveragePooling2D()()
output = tf.keras.layers.Dense(1, activation="sigmoid")(x)

model = tf.keras.Model(inputs=base_model.input, outputs=output)

for layer in base_model.layers:
    layer.trainable = False # Esto impide que las capas se re entrenen

In [ ]:
# # Transfer Learning
# base_model = tf.keras.applications.vgg16.VGG16(include_top=False, weights='imagenet', input_shape=(256, 256, 3))
# avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
# output = tf.keras.layers.Dense(1, activation="sigmoid")(avg)

# model = tf.keras.Model(inputs=base_model.input, outputs=output)

# for layer in base_model.layers:
#     layer.trainable = False # Esto impide que las capas se re entrenen

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(patience = 7)

METRICS = [
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
      tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]


model.compile(loss = 'bce', optimizer = 'adam', metrics=METRICS)
history_resnet = model.fit(X_train_c, y_train, epochs=1000, 
                        validation_data=(X_val_c, y_val),
                        batch_size=10,
                        callbacks=[early_stopping])

In [ ]:
result = model.evaluate(X_test_c[:2000],  y_test[:2000], verbose=2, batch_size=4)